In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
#from keras.utils.np_utils import to_categorical
#from keras.models import Sequential
#from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
#import keras.backend as K
import glob
from PIL import Image
from sklearn.model_selection import train_test_split 
#from keras.optimizers import SGD, Adam, Adadelta, RMSprop
from sklearn.metrics import confusion_matrix
#from keras.callbacks import EarlyStopping
from keras.preprocessing import image
import cv2
import os
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from skimage import io
import labelme
from Img_pro_defs import *
PIXEL = 1080 * 1616
from scipy import ndimage, misc

Using TensorFlow backend.


In [11]:
files = glob.glob('Data/Originals/starco_white_half/*.JPG')
files = sorted_nicely(files)
src = glob.glob('Data/canola/test_t/starco/predicted/*.png')
src = sorted_nicely(src)
path = 'Data/canola/test_t/starco/reconst'

In [12]:
files[0][33:53]

'PICT_20190531_102135'

In [127]:
data =  pd.read_csv('Data/thesis/unet.csv')
files = pd.DataFrame()
files['loss'] = 0
files['acc'] = 0
files['val_loss'] = 0
files['val_acc'] = 0
data['Training'][0]


'512/512 [==============================] - 126s 247ms/step - loss: 0.1269 - acc: 0.9536 - val_loss: 0.0704 - val_acc: 0.9754'

In [128]:
loss = []
acc = []
val_loss = []
val_acc = []
for k in range (len(data)):
    hold = data['Training'][k]
    loss.append(float(hold[67:73]))
    acc.append(float(hold[81:87]))
    val_loss.append(float(hold[100:106]))
    val_acc.append(float(hold[118:]))
   

In [129]:
files['loss'] = loss
files['acc'] = acc
files['val_loss'] = val_loss
files['val_acc'] = val_acc

In [130]:
files

,loss,acc,val_loss,val_acc
0,0.1269,0.9536,0.0704,0.9754
1,0.0499,0.9825,0.0482,0.9822
2,0.0491,0.9826,0.0473,0.9816
3,0.0491,0.9821,0.0442,0.9830
4,0.0420,0.9850,0.0423,0.9842
5,0.0440,0.9842,0.0441,0.9836
6,0.0422,0.9849,0.0374,0.9864
7,0.0382,0.9863,0.0414,0.9841
8,0.0410,0.9852,0.0457,0.9828
9,0.0404,0.9856,0.0404,0.9844


In [132]:
files.to_csv('Data/thesis/unet_t.csv')

In [13]:
f_names = []
for k,file in enumerate(files):
    f_names.append(file[33:53])
    
dest = []
for k, file in enumerate (f_names):
    dest.append (os.path.join(path,'{}.png'.format(file)))
    
image_reconst (src,dest)
    

In [8]:
dest

['Data/canola/temp_test/starco/reconst/PICT_20190531_102135.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102150.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102154.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102159.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102208.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102213.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102228.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102233.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102242.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102247.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102251.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102256.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102302.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102312.png',
 'Data/canola/temp_test/starco/reconst/PICT_20190531_102318.png',
 'Data/can

In [22]:
f_names = []
for k,file in enumerate(files):
    f_names.append(file[:-4])



In [25]:
dest = []
for k, file in enumerate (f_names):
    dest.append (os.path.join(path,'{}.png'.format(file)))

In [33]:
dest[0][44:64]

'PICT_20190610_094429'

In [9]:
src = glob.glob('Data/thesis/temp_test/engels/segnet/reconst/*png')
src = sorted_nicely(src)


In [10]:
src = np.array([image.img_to_array(image.load_img(fname,target_size = (1080,1600))) for fname in src])

In [11]:
del src

In [62]:
x = x.sum(axis=2)


In [66]:
len(x[x!=432])/(PIXEL)*100

1.1490419875320865

In [59]:
len(x[x<0.5])

1728000

In [49]:
data = pd.read_csv('Data/thesis/temp_test/engels/engels.csv')
#files = csv_file['filename']
#files = ['Data/Back_rm/Fred_204_205_N/' + w for w in files]
data = data.set_index('filename')
data

,lat,lon,Zone ID,Crop Leaves Area,Weed Leaves Area
filename,,,,,
PICT_20190610_094429.JPG,52.475185,-104.643150,1,NaN,NaN
PICT_20190610_094728.JPG,52.478294,-104.642807,1,NaN,NaN
PICT_20190610_094732.JPG,52.478279,-104.642227,1,NaN,NaN
PICT_20190610_094740.JPG,52.478264,-104.641960,1,NaN,NaN
PICT_20190610_100207.JPG,52.473339,-104.641579,1,NaN,NaN
PICT_20190610_100910.JPG,52.475632,-104.639610,1,NaN,NaN
PICT_20190610_100915.JPG,52.475594,-104.639648,1,NaN,NaN
PICT_20190610_100920.JPG,52.475452,-104.639801,1,NaN,NaN
PICT_20190610_101101.JPG,52.473476,-104.641960,1,NaN,NaN


In [46]:
image = cv2.imread (dest[0])

image = image.sum(axis = 2)
image[image!=432] = 1
image[image==432] = 0
weed_percent =sum(image)/(1056*1600)*100
np.sum(image)

8102

In [50]:
#files = glob.glob('Data/thesis/temp_test/engels/segnet/reconst/*png')
#data = crop_pixel_counter (files, csv_file, 20, 3)
#files = glob.glob('Data/temp/fredericks_204_205_N/30_05_2019/*.png')
files = sorted_nicely(files)


for k, file in enumerate (dest):
    image = cv2.imread (file)
    image = image.sum(axis = 2)
    image[image!=432] = 1
    image[image==432] = 0
    weed_percent =np.sum(image)/(1056*1600)*100
    data['Weed Leaves Area'][file[44:64] + '.JPG'] = weed_percent
    #data['Crop Leaves Area'][file[44:64] + '.JPG'] = data['Crop Leaves Area'][files[k][42:62] + '.JPG'] - weed_percent
    
#data.to_csv('Data/CSV/fredericks_204_205_N/Fredericks 204_205_North.csv')    
data





        
    

/home/asad/anaconda3/envs/py360/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,lat,lon,Zone ID,Crop Leaves Area,Weed Leaves Area
filename,,,,,
PICT_20190610_094429.JPG,52.475185,-104.643150,1,NaN,0.479522
PICT_20190610_094728.JPG,52.478294,-104.642807,1,NaN,6.283026
PICT_20190610_094732.JPG,52.478279,-104.642227,1,NaN,0.743253
PICT_20190610_094740.JPG,52.478264,-104.641960,1,NaN,5.510772
PICT_20190610_100207.JPG,52.473339,-104.641579,1,NaN,0.258523
PICT_20190610_100910.JPG,52.475632,-104.639610,1,NaN,0.237334
PICT_20190610_100915.JPG,52.475594,-104.639648,1,NaN,0.524266
PICT_20190610_100920.JPG,52.475452,-104.639801,1,NaN,0.097538
PICT_20190610_101101.JPG,52.473476,-104.641960,1,NaN,0.222183


In [51]:
data.to_csv('Data/thesis/temp_test/engels/engels_f.csv')  

In [13]:
files = glob.glob('Data/temp/fredericks_204_205_N/30_05_2019/*.png')
files[0][42:62]

'PICT_20190603_142900'

In [3]:

#files = glob.glob('Data/Back_rm/Fred_204_205_N/*.JPG')
data = crop_pixel_counter (files, csv_file, 20, 3)
#files = glob.glob('Data/temp/fredericks_204_205/30_05_2019/*.png')
files = sorted_nicely(files)
data



/home/asad/CropPro Project/Img_pro_defs.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['Crop Leaves Area'][k] = crop_percent


,lat,lon,Zone ID,Crop Leaves Area,Weed Leaves Area
filename,,,,,
PICT_20190603_141756.JPG,52.065754,-104.393532,6,27.341630,0.0
PICT_20190603_141806.JPG,52.065750,-104.393707,6,18.297064,0.0
PICT_20190603_141811.JPG,52.065742,-104.394127,4,10.800387,0.0
PICT_20190603_141819.JPG,52.065739,-104.394348,4,5.553092,0.0
PICT_20190603_141824.JPG,52.065735,-104.394592,4,1.829506,0.0
PICT_20190603_141831.JPG,52.065727,-104.394836,4,5.666140,0.0
PICT_20190603_141931.JPG,52.065704,-104.395332,5,5.144447,0.0
PICT_20190603_141943.JPG,52.065701,-104.395500,5,2.818688,0.0
PICT_20190603_141949.JPG,52.065685,-104.395859,4,4.044394,0.0


In [5]:
data.to_csv('Data/CSV/fredericks_204_205_/Fredericks 204-205 North.csv')

In [ ]:
data.loc['PICT_20190530_150019.JPG']

In [6]:
image = cv2.imread ('Data/Back_rm/Fred_204_205/30-05-2019/PICT_20190530_142058.JPG')

In [108]:
image1 = cv2.imread ('Data/temp/fredericks_204_205/30_05_2019/PICT_20190530_142058.png')

In [21]:
image = cv2.imread ('Data/Back_rm/Fred_204_205/30-05-2019/PICT_20190530_152239.JPG')
image = thresholding(image,20)
image = median_filter(image,5)
image[image!=0] = 1
image.sum()

293707

In [22]:
cv2.imwrite('result2.png',255*image)

True

True

True

In [82]:
my_image.sum()/PIXEL*100

9.868387880454712

In [10]:
from PIL import Image

img = Image.open('Data/temp/fredericks_204_205/30_05_2019/PICT_20190530_142110.png')

In [11]:
arr = (np.asarray(img))
s = arr.sum(-1)

In [12]:
s[s!=0]=1
s.sum()/PIXEL*100

0.6106183535020169

In [23]:
files = glob.glob('Data/temp/fredericks_204_205/30_05_2019/*.png')


In [ ]:
files = glob.glob('Data/temp/fredericks_204_205/30_05_2019/*.png')
files = sorted_nicely(files)
for k, file in enumerate (files):
    index = 
    image = cv2.imread (file)
    image[image != 0] = 1
    crop_percent = image.sum()/PIXEL*100
    data['Crop %age'][files[0][40:60] + '.JPG'] = crop_percent/3

In [ ]:
def weed_pixel_counter (files, data):
    files = sorted_nicely(files)
    data = data.set_index('filename')
    data['Crop %age'] = 0
    data['Weed %age'] = 0
    for k, file in enumerate (files):
        image = cv2.imread (file)
        image[image != 0] = 1
        crop_percent = image.sum()/PIXEL*100
        data['Crop %age'][k] = crop_percent
    return data